In [1]:
! pip install --upgrade google-cloud-bigquery-storage

     |████████████████████████████████| 125 kB 6.2 MB/s eta 0:00:01


In [27]:
import google, pathlib, shutil, urllib, zipfile as zf, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from shapely.ops import orient
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
root_path = pathlib.Path(root_path)
raw_data_path = root_path / 'raw_data'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

def read_file(zipfile, f, **kwargs):
    file = zipfile.extract(f)
    df = pd.read_csv(file, dtype=str, **kwargs)
    df.columns = [str(x).lower() for x in df.columns]
    return df

def get_states():
    qry = f"""
    select
        state_fips_code as fips
        , state_postal_abbreviation as abbr
        , state_name as name
    from
        bigquery-public-data.census_utility.fips_codes_states
    where
        state_fips_code <= '56'
    """
    return bqclient.query(qry).result().to_dataframe()

def get_data(name, url, state, yr=2020, overwrite=False):
    print(f"\nGet {state['name']} {yr} {name}", end='...' )
    path = f"{name}/{yr}/{state['abbr']}"
    zipfile = raw_data_path / f"{path}/{path.replace('/', '_')}.zip"
    path = zipfile.parent
    
    if overwrite:
        shutil.rmtree(path, ignore_errors=True)
    path.mkdir(parents=True, exist_ok=True)
    %cd {path}
    try:
        zf.ZipFile(zipfile)
        print('already have zip')
    except:
        try:
            print(f'fetching zip from {url}', end = '...')
            zf.ZipFile(urllib.request.urlretrieve(url, zipfile)[0])
            print('success')
        except urllib.error.HTTPError:
            print('\n\nFAILED - BAD URL\n\n')
            zipfile = None
    parquetfile = zipfile.with_suffix('.parquet')
    try:
        df = gpd.read_parquet(parquetfile)
    except:
        try:
            df = pd.read_parquet(parquetfile)
        except:
            df = None
    return zipfile, df

def write_data(df, file):
    df.to_parquet(file.with_suffix('.parquet'))
    table_id = f"{proj_id}.redistricting.{file.stem}"
    bqclient.delete_table(table_id, not_found_ok=True)
    job = bqclient.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for the job to complete.

states = get_states()
states = states.iloc[[39,43]]
overwrite = False
# overwrite = True
get_states

In [ ]:
different colors on different columns change the polygon colors 
how to color a polygon based on a column
control pop up information how to take off stuff and how to add stuff
https://bigquerygeoviz.appspot.com?shareid=Fwpse5POU4s9zeCI5kIn

In [56]:
def read_tbl(tbl, rows=None, start=0):
    query = f'select * from {tbl}'
    if start is not None:
        query += f' offset {start}'
    if rows is not None:
        query += f' fetch {rows}'
    print(query)
    df = bqclient.query(query).result().to_dataframe()
    try:
        df = bqclient.query(query).result().to_dataframe()
        display(df)
        return df
    except:
        return None

In [70]:
def bg_2010_RI():
    qry = f"""
    select 
        *
    from
        cmat-315920.redistricting_data.tabblock_2010_RI
    --where
        --row<=10
    order by geoid_2010
    limit 15
    --offset 10 
    """
    df = bqclient.query(qry).result().to_dataframe()
    return df
bg_2010_RI()

,geoid_2010,aland,perim,polsby_popper,centroid,geometry,geometry_simple
0,440010301001000,7996,403.623442,0.616779,b'\x01\x01\x00\x00\x00)\xa3\x80\xa3S\xd5Q\xc0\...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0f\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
1,440010301001001,11821,468.511019,0.676744,b'\x01\x01\x00\x00\x00;\xacp\xcbG\xd5Q\xc0 *\x...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x17\x00...","b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00..."
2,440010301001002,6001,321.325951,0.730368,b'\x01\x01\x00\x00\x00\x90\xb9\xd7\xeeC\xd5Q\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x10\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
3,440010301001003,16049,582.946864,0.593472,b'\x01\x01\x00\x00\x00\xaf\x82s\xa1M\xd5Q\xc0q...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x1b\x00...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00..."
4,440010301001004,8264,404.426583,0.634923,b'\x01\x01\x00\x00\x00\xc4\xaa\xe6\xdeh\xd5Q\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
5,440010301001005,10972,485.785189,0.584261,b'\x01\x01\x00\x00\x00\x06\xa6.t\x80\xd5Q\xc0\...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x13\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
6,440010301001006,16672,539.989934,0.718499,b'\x01\x01\x00\x00\x00\x81{\x9e?m\xd5Q\xc0-\x8...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x19\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
7,440010301001007,17056,600.128554,0.595112,"b""\x01\x01\x00\x00\x00\xc0\xc2\x93'Z\xd5Q\xc0\...","b'\x01\x03\x00\x00\x00\x01\x00\x00\x00""\x00\x0...",b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
8,440010301001008,313902,2966.603017,0.448214,b'\x01\x01\x00\x00\x00X\x81k8v\xd5Q\xc0E\x99\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00o\x00\x0...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
9,440010301001009,15593,498.989814,0.786966,b'\x01\x01\x00\x00\x00\xee\n}\xb0\x8c\xd5Q\xc0...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x1e\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...


In [74]:
def tab_2010_RI():
    qry = f"""
    select 
        *
    from
        cmat-315920.redistricting_data.tabblock_2010_RI
    order by geoid_2010
    limit 15
    offset 10 
    """
    df = bqclient.query(qry).result().to_dataframe()
    return df
tab_2010_RI()

,geoid_2010,aland,perim,polsby_popper,centroid,geometry,geometry_simple
0,440010301001010,26091,771.679720,0.550588,b'\x01\x01\x00\x00\x00`}\x97\xad\xd0\xd5Q\xc0u...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
1,440010301001011,19089,1059.035943,0.213881,b'\x01\x01\x00\x00\x00\x00f\x08\x11\xba\xd5Q\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00/\x00\x0...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
2,440010301001012,2086,313.686237,0.266399,b'\x01\x01\x00\x00\x00\xff\x14}\xf4\xbc\xd5Q\x...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x16\x00...",b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
3,440010301001013,592841,5625.224906,0.235434,"b'\x01\x01\x00\x00\x00>\xa3\xbevz\xd5Q\xc0}(,L...",b'\x01\x03\x00\x00\x00\x02\x00\x00\x00\x8f\x00...,b'\x01\x03\x00\x00\x00\x02\x00\x00\x00\x06\x00...
4,440010301001014,18804,551.287907,0.777505,b'\x01\x01\x00\x00\x00\xfa\xb5\xf5\xd3\x7f\xd5...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0f\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
5,440010301001015,10019,419.810301,0.714378,b'\x01\x01\x00\x00\x00\xe6\x97\x1c\xd2\x83\xd5...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
6,440010301001016,13035,498.188868,0.659983,b'\x01\x01\x00\x00\x00\xeb\xfc\xdbe?\xd5Q\xc0*...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
7,440010301001017,10178,449.597953,0.632738,b'\x01\x01\x00\x00\x00H\x9f\xfb\x17\xad\xd5Q\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
8,440010301001018,12489,480.577018,0.679535,b'\x01\x01\x00\x00\x00\x94\xbe\x10r^\xd5Q\xc0\...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\t\x00\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
9,440010301001019,11967,469.019636,0.683617,b'\x01\x01\x00\x00\x00\x92b\xdb\xfd\x85\xd5Q\x...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x11\x00...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...


In [ ]:
from google.cloud.bigquery_storage import BigQueryReadClient
from google.cloud.bigquery_storage import types

client = BigQueryReadClient()
table = cmat-315920.redistricting_data.tabblock_2010_RI

requested_session = types.ReadSession()
requested_session.table = table

parent = "projects/{}".format(project_id)
session = client.create_read_session(
    parent=parent,
    read_session=requested_session,
    max_stream_count=1,
)
reader = client.read_rows(session.streams[0].name)


rows = reader.rows(session)

names = set()
states = set()

for row in rows:
    names.add(row["name"])
    states.add(row["state"])

print("Got {} unique names in states: {}".format(len(names), states))




In [83]:
def tab_2020_TX():
    qry = f"""
    select 
        *
    from
        cmat-315920.redistricting.tabblock_2020_TX
    order by geoid_20
    limit 100000
    offset 1000 
    """
    df = bqclient.query(qry).result().to_dataframe()
    return df
tab_2020_TX()
#!bq help

,geoid_20,geometry,lon,lat,area,geometry_simple,perim,polsby_popper
0,480019507003073,"POLYGON ((-95.632814 31.754593, -95.632452 31....",+31.7542330,-095.6320111,15987,"POLYGON ((-95.632814 31.754593, -95.631418 31....",501.195005,0.799767
1,480019507003074,"POLYGON ((-95.636686 31.756137, -95.636538 31....",+31.7535501,-095.6346700,196237,"POLYGON ((-95.636686 31.756137, -95.6347799999...",2039.271262,0.592981
2,480019507003075,"POLYGON ((-95.63731 31.752434, -95.63587299999...",+31.7520071,-095.6363830,20703,"POLYGON ((-95.63731 31.752434, -95.63587299999...",576.383228,0.783106
3,480019507003076,"POLYGON ((-95.63691 31.751186, -95.635441 31.7...",+31.7509819,-095.6360485,13692,"POLYGON ((-95.63691 31.751186, -95.635441 31.7...",483.523955,0.735937
4,480019507003077,"POLYGON ((-95.638238 31.755134, -95.637362 31....",+31.7546337,-095.6373199,24143,"POLYGON ((-95.638238 31.755134, -95.637362 31....",620.956353,0.786826
...,...,...,...,...,...,...,...,...
99995,480770302003040,"POLYGON ((-98.185079 33.81721, -98.184162 33.8...",+33.8167305,-098.1844399,11746,"POLYGON ((-98.185079 33.81721, -98.18399599999...",444.742752,0.746246
99996,480770302003041,"POLYGON ((-98.18399599999999 33.817251, -98.18...",+33.8168559,-098.1828778,21220,"POLYGON ((-98.18399599999999 33.817251, -98.18...",593.336809,0.757448
99997,480770302003042,"POLYGON ((-98.181873 33.818039, -98.1816099999...",+33.8169667,-098.1812938,22093,"POLYGON ((-98.181873 33.818039, -98.1808039999...",680.576089,0.599392
99998,480770302003043,"POLYGON ((-98.18080399999999 33.818032, -98.17...",+33.8170134,-098.1796238,51536,"POLYGON ((-98.18080399999999 33.818032, -98.17...",914.362639,0.774611


In [ ]:
SELECT
    A.*,
    B.geography,
    B.point
FROM
    `cmat-315920.redistricting_data.nodes_tract_2020_TX` as A
join
    `cmat-315920.redistricting_data.shapes_tract_2020_TX` as B
on
    A.geoid_2020 = B.geoid_2020


In [7]:
def get_crosswalks(state, overwrite=False):
    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{state['fips']}.zip"
    yr = 2020
    zipfile, df = get_data(name, url, state, yr, overwrite)
    
    if df is None and zipfile is not None:
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            if file[-3:] == 'txt':
                df = read_file(Z, file, sep='|')
                for yr in [2010, 2020]:
                    df[f'geoid_{yr}'] = df[f'state_{yr}'].str.rjust(2,'0') + df[f'county_{yr}'].str.rjust(3,'0') + df[f'tract_{yr}'].str.rjust(6,'0') + df[f'blk_{yr}'].str.rjust(4,'0')
                df = df[['geoid_2010', 'geoid_2020']]
        write_data(df, zipfile)
    return df

def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            col = file.lower().split('_')[-1][:-4]
            if file[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(Z, file, sep=sep)
                if col == 'vtd':
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        write_data(df, zipfile)
    return df

def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            w = file.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(Z, file)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        write_data(df, zipfile)
    return df

def get_census(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            #     source_format=bigquery.SourceFormat.CSV,
        )
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            table_id = f"{proj_id}.redistricting.{zipfile.stem}_"
            print(file)
            if file[2:5] == 'geo':
                Z.extract(file)
                temp_table_id = table_id + 'temp'
                with open(file, mode="rb") as f:
                    load_job = bqclient.load_table_from_file(f, temp_table_id, job_config=job_config)
                    load_job.result()

                widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
                names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                sel = [f'trim(substring(string_field_0, {s}, {w})) as {n}' for s, w, n in zip(starts, widths, names)]
                qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id
                
                bqclient.delete_table(table_id+'geo', not_found_ok=True)
                query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id+'geo', autodetect=True))
                query_job.result()
                bqclient.delete_table(temp_table_id)
            else:
                names = [['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],
                         ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18',
                          'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']]
                i = file[6]
                if i in ['1', '2']:
                    Z.extract(file)
                    cmd = 'sed -i "1s/^/' + ','.join(census_columns[int(i)-1]) + '\\n/" ' + file
                    ! {cmd}
                    with open(file, mode="rb") as f:
                        load_job = bqclient.load_table_from_file(f, table_id+i, job_config=job_config)
                        load_job.result()
#         ja = '\n, A.'
#         jb = '\n, B.'
#         qry = f"""
#         select
#             concat(right(concat("00",C.state), 2), right(concat("000",C.county), 3), right(concat("000000",C.tract), 6), right(concat("0000",C.block), 4)) as geoid
#             , C.*
#             , A.{ja.join(names[0][5:])}
#             , B.{jb.join(names[1][5:])}
#         from
#             {table_id}1 as A
#         inner join
#             {table_id}2 as B
#         on
#             A.fileid = B.fileid
#             and A.stusab = B.stusab
#             and A.logrecno = B.logrecno
#         inner join
#             {table_id}geo as C
#         on
#             A.fileid = trim(C.fileid)
#             and A.stusab = trim(C.stusab)
#             and A.logrecno = cast(C.logrecno as int)
#         where
#             C.block != ""
#         order by
#             A.logrecno
#         """
# #         print(qry)
#         bqclient.delete_table(table_id+'comb', not_found_ok=True)
#         query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id+'comb'))
#         query_job.result()
                         
        df = pd.DataFrame()
        df['A'] = [1]
        df.to_parquet(zipfile.with_suffix('.parquet'))
    return df

def join(state, yr):
    print(f"\nJoining {state['name']} {yr}", end='...' )
    def table_id(name):
        return f"{proj_id}.redistricting.{name}_{yr}_{state['abbr']}"
    ja = '\n, A.'
    jb = '\n, B.'
    qry = f"""
    select
        E.cntyvtd
        , sum(total) over (partition by cntyvtd) as cntyvtd_pop
        --, total / cntyvtd_pop as cntyvtd_pop_prop
        --, total / sum(total) over (partition by cntyvtd) as cntyvtd_pop_prop
        , D.*
    from (
        select
            concat(right(concat("00",C.state), 2), right(concat("000",C.county), 3), right(concat("000000",C.tract), 6), right(concat("0000",C.block), 4)) as geoid
            , C.*
            , A.{ja.join(census_columns[0][5:])}
            , B.{jb.join(census_columns[1][5:])}
        from
            {table_id('census')}_1 as A
        inner join
            {table_id('census')}_2 as B
        on
            A.fileid = B.fileid
            and A.stusab = B.stusab
            and A.logrecno = B.logrecno
        inner join
            {table_id('census')}_geo as C
        on
            A.fileid = trim(C.fileid)
            and A.stusab = trim(C.stusab)
            and A.logrecno = cast(C.logrecno as int)
        where
            C.block != ""
        ) as D
    inner join
        {table_id('assignments')} as E
    on
        D.geoid = E.geoid
    """
#         print(qry)
    bqclient.delete_table(table_id('comb'), not_found_ok=True)
    query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id('comb')))
    query_job.result()
    print('success')
                         
census_columns = [['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],
                  ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18',
                   'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']]



elections = get_elections_TX(overwrite)
for i, state in states.iterrows():
    crosswalks = get_crosswalks(state, overwrite)
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)
        if yr == 2010:
            get_census(state, yr, overwrite)
            join(state, yr)


Get Texas 2020 elections.../home/jupyter/raw_data/elections/2020/TX
already have zip

Get Rhode Island 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/RI
already have zip

Get Rhode Island 2020 assignments.../home/jupyter/raw_data/assignments/2020/RI
already have zip

Get Rhode Island 2010 assignments.../home/jupyter/raw_data/assignments/2010/RI
already have zip

Get Rhode Island 2010 census.../home/jupyter/raw_data/census/2010/RI
already have zip

Joining Rhode Island 2010...success

Get Texas 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/TX
already have zip

Get Texas 2020 assignments.../home/jupyter/raw_data/assignments/2020/TX
already have zip

Get Texas 2010 assignments.../home/jupyter/raw_data/assignments/2010/TX
already have zip

Get Texas 2010 census.../home/jupyter/raw_data/census/2010/TX
already have zip

Joining Texas 2010...success


In [ ]:
10% population deviation


In [ ]:
make new tables from sum of blocks in this vtd every block will have total pop in that vtd lots of repitetion
find that live in vtd and add up total population using the cntyvtd

using comb 2010 tx
2. find proportion of 
divide table 1 by total and so on 
3. total votes by calc in table 2 multipy
join two tables together join with elections
window function

In [ ]:
# def get_census_2010():
def get_census(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            #     source_format=bigquery.SourceFormat.CSV,
        )
        Z = zf.ZipFile(zipfile)
        for file in Z.namelist():
            table_id = f"{proj_id}.redistricting.{zipfile.stem}_"
            print(file)
            if file[2:5] == 'geo':
                Z.extract(file)
                temp_table_id = table_id + 'temp'
                with open(file, mode="rb") as f:
                    load_job = bqclient.load_table_from_file(f, temp_table_id, job_config=job_config)
                    load_job.result()

                widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
                names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                sel = [f'substring(string_field_0, {s}, {w}) as {n}' for s, w, n in zip(starts, widths, names)]
                qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id
#                 print(qry)
                
                table_id += 'geo'
                query_job = bqclient.query(qry, job_config=bigquery.QueryJobConfig(destination=table_id))
                query_job.result()
#                 bqclient.delete_table(temp_table_id)
            else:
                names = [['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl'],
                         ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']]
                i = file[6]
                if i in ['1', '2']:
                     Z.extract(file)
                     cmd = 'sed -i "1s/^/' + ','.join(names[int(i)-1]) + '\\n/" ' + file
                     ! {cmd}
                     table_id += i
                     with open(file, mode="rb") as f:
                         load_job = bqclient.load_table_from_file(f, table_id, job_config=job_config)
                         load_job.result()
for i, state in states.iterrows():
    for yr in [2010]:#, 2020]:
        census = get_census(state, yr, overwrite)

In [ ]:
temp_table_id = f"{proj_id}.cook_test.temp"

widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
starts = np.insert(np.cumsum(widths)[:-1], 0 ,0) + 1
names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
sel = [f'substring(string_field_0, {s}, {w}) as {n}' for s, w, n in zip(starts, widths, names)]
qry = 'select\n\t' + ',\n\t'.join(sel) + '\nfrom\n\t' + temp_table_id

table_id = f"{proj_id}.cook_test.{zipfile.stem}{geo}"
job_config = bigquery.QueryJobConfig(destination=table_id)
query_job = client.query(qry, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print(qry)

In [ ]:
s = [1,2,3]
np.insert(np.cumsum(s)[:-1], 0 ,0)+1
np.cumsum(s)
# np.insert?

In [ ]:
L = ['1', '2']
L.index

In [ ]:
s = 'ri000012010.pl'
s[6] == 1

In [ ]:
path = raw_data_path / 'census/2010/RI'
file = path /'ri000012010.pl'

table_id = f"{proj_id}.cook_test.{file.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

%cd {file.parent}
names1 = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_hl', 'hispanic_or_latino_hl', 'not_hispanic_or_latino_hl', 'population_of_one_race_hl', 'white_alone_hl', 'black_or_african_american_alone_hl', 'american_indian_and_alaska_native_alone_hl', 'asian_alone_hl', 'native_hawaiian_and_other_pacific_islander_alone_hl', 'some_other_race_alone_hl', 'population_of_two_or_more_races_hl', 'population_of_two_races_hl', 'white_black_or_african_american_hl', 'white_american_indian_and_alaska_native_hl', 'white_asian_hl', 'white_native_hawaiian_and_other_pacific_islander_hl', 'white_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_hl', 'black_or_african_american_asian_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_some_other_race_hl', 'american_indian_and_alaska_native_asian_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_hl', 'asian_some_other_race_hl', 'native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_three_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_hl', 'white_black_or_african_american_asian_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_asian_some_other_race_hl', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_asian_some_other_race_hl', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'american_indian_and_alaska_native_asian_some_other_race_hl', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_four_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_asian_some_other_race_hl', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_five_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_hl', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl', 'population_of_six_races_hl', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_hl']
names2 = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total_18', 'population_of_one_race_18', 'white_alone_18', 'black_or_african_american_alone_18', 'american_indian_and_alaska_native_alone_18', 'asian_alone_18', 'native_hawaiian_and_other_pacific_islander_alone_18', 'some_other_race_alone_18', 'population_of_two_or_more_races_18', 'population_of_two_races_18', 'white__black_or_african_american_18', 'white__american_indian_and_alaska_native_18', 'white__asian_18', 'white__native_hawaiian_and_other_pacific_islander_18', 'white__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native_18', 'black_or_african_american__asian_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__some_other_race_18', 'american_indian_and_alaska_native__asian_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander_18', 'asian__some_other_race_18', 'native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_three_races_18', 'white__black_or_african_american__american_indian_and_alaska_native_18', 'white__black_or_african_american__asian_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__some_other_race_18', 'white__american_indian_and_alaska_native__asian_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander_18', 'white__asian__some_other_race_18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__asian__some_other_race_18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'american_indian_and_alaska_native__asian__some_other_race_18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_four_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__asian__some_other_race_18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__american_indian_and_alaska_native__asian__some_other_race_18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_five_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'population_of_six_races_18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_18', 'total_hl18', 'hispanic_or_latino_hl18', 'not_hispanic_or_latino_hl18', 'population_of_one_race_hl18', 'white_alone_hl18', 'black_or_african_american_alone_hl18', 'american_indian_and_alaska_native_alone_hl18', 'asian_alone_hl18', 'native_hawaiian_and_other_pacific_islander_alone_hl18', 'some_other_race_alone_hl18', 'population_of_two_or_more_races_hl18', 'population_of_two_races_hl18', 'white__black_or_african_american_hl18', 'white__american_indian_and_alaska_native_hl18', 'white__asian_hl18', 'white__native_hawaiian_and_other_pacific_islander_hl18', 'white__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native_hl18', 'black_or_african_american__asian_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__some_other_race_hl18', 'american_indian_and_alaska_native__asian_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander_hl18', 'asian__some_other_race_hl18', 'native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_three_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native_hl18', 'white__black_or_african_american__asian_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__asian__some_other_race_hl18', 'white__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__asian__some_other_race_hl18', 'black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'american_indian_and_alaska_native__asian__some_other_race_hl18', 'american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_four_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__asian__some_other_race_hl18', 'white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_five_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__some_other_race_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'population_of_six_races_hl18', 'white__black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander__some_other_race_hl18', 'housing_total', 'housing_occupied', 'housing_vacant']
cmd = 'sed -i "1s/^/' + ','.join(names) + '\\n/" ' + file.name
! {cmd}

In [ ]:
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population_of_one_race', 'white_alone', 'black_or_african_american_alone', 'american_indian_and_alaska_native_alone', 'asian_alone', 'native_hawaiian_and_other_pacific_islander_alone', 'some_other_race_alone', 'population_of_two_or_more_races', 'population_of_two_races', 'white_black_or_african_american', 'white_american_indian_and_alaska_native', 'white_asian', 'white_native_hawaiian_and_other_pacific_islander', 'white_some_other_race', 'black_or_african_american_american_indian_and_alaska_native', 'black_or_african_american_asian', 'black_or_african_american_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_some_other_race', 'american_indian_and_alaska_native_asian', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander', 'asian_some_other_race', 'native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_three_races', 'white_black_or_african_american_american_indian_and_alaska_native', 'white_black_or_african_american_asian', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_some_other_race', 'white_american_indian_and_alaska_native_asian', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander', 'white_asian_some_other_race', 'white_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_asian_some_other_race', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'american_indian_and_alaska_native_asian_some_other_race', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_four_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_asian_some_other_race', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_american_indian_and_alaska_native_asian_some_other_race', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_five_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'population_of_six_races', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_HL', 'hispanic_or_latino_HL', 'not_hispanic_or_latino_HL', 'population_of_one_race_HL', 'white_alone_HL', 'black_or_african_american_alone_HL', 'american_indian_and_alaska_native_alone_HL', 'asian_alone_HL', 'native_hawaiian_and_other_pacific_islander_alone_HL', 'some_other_race_alone_HL', 'population_of_two_or_more_races_HL', 'population_of_two_races_HL', 'white_black_or_african_american_HL', 'white_american_indian_and_alaska_native_HL', 'white_asian_HL', 'white_native_hawaiian_and_other_pacific_islander_HL', 'white_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_HL', 'black_or_african_american_asian_HL', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_some_other_race_HL', 'american_indian_and_alaska_native_asian_HL', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'american_indian_and_alaska_native_some_other_race_HL', 'asian_native_hawaiian_and_other_pacific_islander_HL', 'asian_some_other_race_HL', 'native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_three_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_HL', 'white_black_or_african_american_asian_HL', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_HL', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'white_american_indian_and_alaska_native_some_other_race_HL', 'white_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_asian_some_other_race_HL', 'white_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_HL', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_american_indian_and_alaska_native_some_other_race_HL', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_asian_some_other_race_HL', 'black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'american_indian_and_alaska_native_asian_some_other_race_HL', 'american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_four_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_HL', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_american_indian_and_alaska_native_some_other_race_HL', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_asian_some_other_race_HL', 'white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_american_indian_and_alaska_native_asian_some_other_race_HL', 'white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_five_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race_HL', 'white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL', 'population_of_six_races_HL', 'white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race_HL']
names = [c.replace(' ', '_').replace(';', '_').lower() for c in names]
print(names)

In [ ]:
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
#     source_format=bigquery.SourceFormat.CSV,
)

with open(file, mode="rb") as f:
    load_job = bqclient.load_table_from_file(f, table_id, job_config=job_config)
    load_job.result()

In [ ]:
# df = pd.read_csv(file, nrows=100)
# # df.columns = L
# # df.isnull().any().any()
# # df.shape
# job = bqclient.load_table_from_dataframe(df, table_id)
# job.result()  # Wait for the job to complete.

# bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)


with open(file, mode="rb") as f:
    load_job = bqclient.load_table_from_file(
        f, table_id,
        job_config=job_config
    )
    load_job.result() 
#     df = pd.read_csv(f, nrows=2, header=None)

In [ ]:
df = pd.read_csv(file, nrows=100)
df

In [ ]:
df.dtypes

In [ ]:
bigquery.LoadJobConfig??

In [ ]:
t = bqclient.get_table(table_id)
df = bqclient.list_rows(t).to_dataframe()

In [ ]:
df.head(10)
# df['int64_field_5'].unique()

In [ ]:
names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
cmd = 'sed -i "1s/^/' + ','.join(names) + '\n/"' + 
cmd


In [ ]:
df = pd.read_csv(file, nrows=10)
df

In [ ]:
file

In [ ]:
%ls
table_id = f"{proj_id}.cook_test.{parquetfile.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 

In [ ]:
# zf.extract_all(zipfile)

# zipfile.extractall()
for f in zipfile.namelist():
    if f[-2:] == 'pl':
#         file = zipfile.extract(f)
        if f[2:5] == 'geo':
            pass
#             widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
#             names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
#             df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
#             df = df.query('block != ""')
        elif f[6] == '1':
            names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
            cmd = 'sed -i "1s/^/' + ','.join(names) + '\\n/" ' + file
            print(cmd)
            ! {cmd}
#             names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
#             df = read_file(zipfile, f          , header=None, nrows=nrows)
        elif f[6] == '2':
            pass
#             names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
#             df = read_file(zipfile, f          , header=None, nrows=nrows)



#     widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
#     names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
#     df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
#     df = df.query('block != ""')
# elif f[6] == '1':
#     names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
#     df = read_file(zipfile, f          , header=None, nrows=nrows)
# elif f[6] == '2':
#     names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
#     df = read_file(zipfile, f          , header=None, nrows=nrows)



In [ ]:
def get_crosswalks(state, overwrite=False):
    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{state['fips']}.zip"
    yr = 2020
    zipfile, df = get_data(name, url, state, yr, overwrite)
    if df is None and zipfile is not None:
        for f in zipfile.namelist():
            if f[-3:] == 'txt':
                df = read_file(zipfile, f, sep='|')
                for yr in [2010, 2020]:
                    df[f'geoid_{yr}'] = df[f'state_{yr}'].str.rjust(2,'0') + df[f'county_{yr}'].str.rjust(3,'0') + df[f'tract_{yr}'].str.rjust(6,'0') + df[f'blk_{yr}'].str.rjust(4,'0')
                df = df[['geoid_2010', 'geoid_2020']]
                display(df.head(2))
        write_data(df, zipfile)
    return df

def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            col = f.lower().split('_')[-1][:-4]
            print(col)
            if f[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(zipfile, f, sep=sep)
                
                if col == 'vtd':
                    display(df.head(2))
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        display(df.head(5))
        write_data(df, zipfile)
    return df

def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    zipfile, df = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            w = f.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(zipfile, f)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, zipfile)
    return df

elections = get_elections_TX(overwrite)
for i, state in states.iterrows():
    crosswalks = get_crosswalks(state, overwrite)
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)

In [ ]:
    job = bqclient.load_table_from_dataframe(df, table_id)

In [ ]:
%ls
table_id = f"{proj_id}.cook_test.{parquetfile.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 


In [ ]:
url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/2020-general-vtd-election-data.zip'
zipfile = root_path / 'temp/data.zip'
%cd {zipfile.parent}
# z = urllib.request.urlretrieve(url, zipfile)[0]
zipfile = zf.ZipFile(z)
for f in zipfile.namelist():
    w = f.split('_')
    if w.pop(-1) == 'Returns.csv':
        print(f)
    

In [ ]:
for 

In [ ]:
abbr = 'TX'
# file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
# file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}000022010.pl'
# file = raw_data_path / f'elections/2020/{abbr}/elections_2020_TX.parquet'
file = raw_data_path / f'elections/2020/{abbr}/2020_General_Election_Returns.csv'

file.is_file()

df = pd.read_csv(file, nrows=10)
df
display(df)

table_id = f"{proj_id}.redistricting.{file.stem}"
table_id

bqclient.delete_table(table_id, not_found_ok=True)

job_config = bigquery.LoadJobConfig(
    autodetect=True,
#     skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
)
with open(file, mode='rb') as f:
    load_job = bqclient.load_table_from_file(
        f, table_id, job_config=job_config
    )
    load_job.result() 

# bqclient.load_table_from_dataframe(df, table_id)


# with open(file, mode='rb') as f:
#     bqclient.load_table_from_file(f, table_id)
# # bqclient.load_table_from_file?
# # bqclient.load_table_from_uri?

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def get_census(state, yr, overwrite=False, nrows=None):
    yrs = [2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    name = f'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        if yr == 2010:
            idx = ['fileid', 'stusab', 'chariter', 'logrecno']
            for f in zipfile.namelist():
                if f[-2:] == 'pl':
                    if f[2:5] == 'geo':
                        widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                        names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                        df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
                        df = df.query('block != ""')
                    elif f[6] == '1':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    elif f[6] == '2':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    df.columns = names
                    L.append(df.drop(columns='cifsn').set_index(idx))
            df = pd.concat(L, axis=1, how='inner').reset_index()
            df.columns = [c.replace(' ', '_').replace(';', '_') for c in df.columns]
        display(df.head(3))
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2010]:#, 2020]:
        census = get_census(state, yr, overwrite, 100)

In [ ]:
df#.isnull().any()

In [ ]:
# pd.read_gbq()
t = bqclient.get_table(table_id)

In [ ]:
bqclient.get_table(table_id)

In [ ]:
df = bqclient.list_rows(t).to_dataframe()
# bqclient.

In [ ]:
bqclient.create_table?

In [ ]:
bqclient.load_table_from_file?

In [ ]:
# bqclient.load_table_from_file?
bqclient.create_table?

In [ ]:
abbr = 'RI'
file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
file.is_file()


In [ ]:
def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            col = f.lower().split('_')[-1][:-4]
            print(col)
            if f[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(zipfile, f, sep=sep)
                
                if col == 'vtd':
                    display(df.head(2))
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        display(df.head(5))
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)
        display(assignments.head(2))

In [ ]:
def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            w = f.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(zipfile, f)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, parquetfile)
    return df
elections = get_elections_TX(overwrite)
display(elections.head(2))

In [ ]:
def get_shapefiles(state, yr, level, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    levels = ['tract', 'bg', 'tabblock']
    assert level in levels, f'level must be in {levels} - got {level}'
    
    name = f'shapefiles_{level}'
    url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{level.upper()}"
    if yr == 2010:
        url += '/2010'
    elif yr == 2020 and level == 'tabblock':
        url += '20'
    url += f"/tl_{yr}_{state['fips']}_{level}{str(yr)[-2:]}"
    if yr == 2020 and level in ['tract', 'bg']:
        url = url[:-2]
    url += '.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        path = parquetfile.parent
        zipfile.extractall(path)
        df = gpd.read_file(path)
        df.columns = [x[:-2].lower() if x[-2:].isnumeric() else x.lower() for x in df.columns]
        df = df[['geoid', 'aland', 'geometry']]
        df['geometry'] = df['geometry'].apply(lambda p: orient(p, -1))
        write_data(df.to_wkt(), parquetfile)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        for level in ['tract', 'bg']:#, 'tabblock']:
            geo = get_shapefiles(state, yr, level, overwrite)

# old

In [ ]:
def get_geo(state, yr, level):
    yr = str(yr)
    if level == 'blockgroup':
        a = 'bg'
    elif level == 'block':
        a = 'tabblock'
    else:
        a = level

    path = root_path / f"geo/{level}/{yr}/{state['abbr']}"
    path.mkdir(exist_ok=True, parents=True)
    try:
        geo = gpd.read_file(path)
    except:
        %cd {path}
        url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{a.upper()}"
        
        if yr == '2010':
            url += '/2010'
        elif yr == '2020' and level == 'block':
            url += '20'
        url += f"/tl_{yr}_{state['fips']}_{a}{yr[-2:]}"
        if yr == '2020' and level in ['blockgroup', 'tract']:
            url = url[:-2]
        url += '.zip'
        ! wget -O geo.zip {url}
        ! unzip geo.zip
        ! rm geo.zip
        geo = gpd.read_file(path)

    geo = lower_cols(geo)[['geoid', 'aland', 'geometry']].apply(lambda p: orient(p, -1))
    geo.columns = ['geo_id', 'geometry']

    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    job.result()
    return geo


for i, state in states.iterrows():
    print(state['name'])
    for level in ['tract', 'blockgroup', 'block']:
        print(level)
        for yr in [2010, 2020]:
            print(yr)
            geo = get_geo(state, yr, level)

In [ ]:
def read_geo(state, yr, level):
    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    qry = f"""
        select
            *
        from
            {table_id}
        """
#     job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    
#     job.result()  # Wait for the job to complete.
    bgs = bqclient.query(qry).result().to_dataframe()
    bgs['geometry'] = gpd.GeoSeries.from_wkt(bgs['geometry'])
    return bgs
#         table = bqclient.get_table(table_id)  # Make an API request.

state = states.iloc[0]
geo = get_geo(state, yr, level)
geo